<div style='background-color:#f7f7f7; padding-top:30px; padding-left:20px; padding-right:20px; padding-bottom:30px'>
    <center>
        <div style='  display: block;
  font-size: 2em;
  font-weight: bold;  display: block;
  font-size: 2em;
  font-weight: bold;'>ASPA - Bayesian Blocks Algorithm and Its Applications
        </div>
    <center>
        <br>
    <i>Giacomo Di Prima, Giuseppe Viterbo</i></center>
    <center>

# Introduction

The histogram representation is a powerful tool that is widely spread across many different fields, yet it is not an objective method since it heavily depends on the choice made for binning the data.
Due to the different approaches one can use, the resulting distributions generated from the same underlying statistics could highlight different structures modifying the interpretation of the data.
Different methods have been devised to overcome this issue:  for example, Scott’s Rule <a name="cite_note-1"><sup>[1]</sup></a> and the Freedman-Diaconis Rule <a name="cite_note-2"><sup>[2]</sup></a> use the number of entries and a measure of the spread of the distribution to determine the number of bins with fixed width; Knuth’s Rule <a name="cite_note-3"><sup>[3]</sup></a> consider the distribution’s structure to build bins that still have the same width; the “equal population” method enforce that each bin have similar numbers of entries, resulting in varying widths, but the location of the bin edges is chosen arbitrarily.<br>
In contrast with them the Bayesian Block (BB) algorithm <a name="cite_note-4"><sup>[4]</sup></a> <a name="cite_note-5"><sup>[5]</sup></a> allows the bin widths to vary and determines the bin edges based on the structure of the distribution, making it parameter free.<br>




<a name="cite_note-1"></a>1. [^](#cite_note-1) D. W. Scott, Biometrika 66, 605 (1979)<br>
<a name="cite_note-2"></a>2. [^](#cite_note-2) D. Freedman and P. Daiconis, Zeitschrift f¨ur Wahrschein-lichkeitstheorie und verwandte Gebiete 57, 453 (1981)<br>
<a name="cite_note-3"></a>3. [^](#cite_note-3) K. H. Knuth, (2006), arXiv:physics/0605197[physics.data-an]<br>
<a name="cite_note-4"></a>4. [^](#cite_note-4) J. D. Scargle, Studies in Astronomical Time Series Analysis. V. Bayesian Blocks, a New Method to Analyze Structure in Photon Counting Data, 1998, arXiv:astro-ph/9711233.<br>
<a name="cite_note-5"></a>5. [^](#cite_note-5) J. D. Scargle, J. P. Norris, B. Jackson, and J. Chiang, Astrophys. J. 764, 167 (2013), arXiv:1207.5578[astro-ph.IM]<br>


# Bayesian Blocks

Each block (or bin, in the context of histograms) is consistent with a probability distribution with compact support; the entire dataset is represented by this collection of finite probability distribution functions.
In the algorithm implementation each block is modeled with a uniform distribution which defines the so called `Piecewise Constant Model`.
By optimizing a goodness-of-fit statistic which depends only on the input data and a regularization parameter that envelop our prior knowledge, the number of blocks and the edges of the blocks, called `change points`, can be computed. Depending on the kind of data the fitness function changes.

|Data type| Description|
|:---|:---|
|Events (TTE)| Time at which the measurements occured|
|Binned Events| Counts of events in time bins|
|Point Measurements |Measurements of a quasi-continuous observable at a sequence of points in time|

The set of blocks is gapless and non-overlapping, where the first block edge is defined by the first data point, and the last block edge is defined by the last data point. A block can contain between 1 and N data points, where the sum of the contents of all the blocks must equal N . The algorithm relies on the additivity of the fitness function, and thus the fitness of a given set of blocks is equal to the sum of the fitnesses of the individual blocks. The total fitness, $F_{total}$ for a given dataset is:

$$F_{total} = \sum_{i=1}^{K} f(B_i)$$

## Fitness
As stated before, depending of the kind of data the fitness function assumes different forms.


### Events
The fitness function can be obtained starting with the unbinned likelihood known as the Cash statistic <a name="cite_note-6"><sup>[6]</sup></a>:

$$\large \log L(\theta) = \sum_{n} \log M (t_n, \theta) - \int M (t, \theta)dt$$

where M(t, θ) is a model of the time dependence of a signal.
The integral is over the observation interval and is the expected number of events under the model.
Since each $k$-th block is constant, and so has a single parameter $M(t, \theta) = \theta$, the resulting log-likelihood is:

$$\large \log L^{(k)}(\theta) = N^{(k)}\log \theta - \theta T^{(k)}$$

where $N^{(k)}$ is the number of events in the $k$-th block and $T^{(k)}$ is the length of the block.</br>
When $\theta =N^{(k)}/T^{(k)} $ the fitness is maximum, yielding:

$$\large \log L^{(k)}_{\max}(\theta) +  N^{(k)} = N^{(k)}(\log N^{(k)} - \log T^{(k)})$$
 
Since the sum of $N^{(k)}$ is constant over $k$, the term on the left hand side is irrelevant since is model indipendent. Moreover, note that this fitness function is scale invariant $(t \to \alpha t)$.

<a name="cite_note-6"></a>6. [^](#cite_note-6) W., Parameter-Estimation in Astronomy
through Application of the Likelihood Ratio, Astrophysical
Journal.<br>

### Binned Event Data
The expected count in a bin is the product $\lambda \cdot e \cdot W$ of the true event rate $\lambda$ at the detector, a dimensionless exposure factor $e$, and
the width of the bin $W$ . Therefore the likelihood for bin n is given
by the Poisson distribution:

$$\large L_n = \frac{(\lambda e_n W_n)^{N_n}e^{- \lambda e_n W_n}}{N_n !}$$

where $N_n$ is the number of events in bin $n$, $\lambda$ is the actual event
rate in counts per unit time, $e_n$ is the exposure averaged over the
bin, and $W_n$ is the bin width in time units. Defining bin efficiency
as $w_n = e_n W_n$, the likelihood for block $k$ is the product of the
likelihoods of all its bins:


$$\large L^{(k)} = \prod_{n=1}^{M^{(k)}} L_n = \lambda^{N^{(k)}} e^{-\lambda w^{(k)}}$$

where $M^{(k)}$ is the number of bins contained in block $k$,

$$\large w^{(k)} = \sum_{n=1}^{M^{(k)}} w_n$$

is the sum of the bin efficiencies in the block, and

$$\large N^{(k)} = \sum_{n=1}^{M^{(k)}} N_n$$

is the total event count in the block.</br>
Notice that the factor $(e_n W_n)^{N_n}/N_n !$ has been discarted since its product over all the bins in all the blocks is constant.
In the end the equation becomes:

$$\large \log L^{(k)}= N^{(k)}\log \lambda - \lambda w^{(k)}$$

### Point Measurements
A common experimental scenario is to measure a signal $s(t)$ at a sequence of times $t_n, n = 1, 2, . . . , N$, known as sampling, in order to characterize its time dependence.
Inevitable corruption due to observational errors is frequently countered by smoothing the data and/or fitting a model.
As with the other data modes Bayesian Blocks is a different approach to this issue, making use of knowledge of the observational error distribution and avoiding the information loss entailed by smoothing.
We assume tha the measurements at these times are independent of each other, which is to say the errors of observation are statistically independent. </br>
Generally these errors are stochastic and addittive, so the measured time series can be modeled as follow:

$$\large x_n \equiv x(t_n) = s(t_n) + z_n$$

The error $z_n$ at time $t_n$ is known only by its distribution. Let us consider $z_n \sim \mathcal{N}(0, \sigma_n)$. If we model the signal as a constant $s = \lambda$, the likelihoood of the $n$-th measurement is:

$$ \large L_n = \frac{1}{\sigma_n \sqrt{2 \pi}} e^{- \frac{1}{2} \left( \frac{x_n - \lambda}{\sigma_n}\right)^2}$$

So, since we assume the indipendence of the measurements, the block $k$ lielihood is:

$$\large L^{(k)} = \prod_n L_n = \frac{(2 \pi)^{- \frac{N_k}{2}}}{\prod_m \sigma_m} e^{-\frac{1}{2} \sum_n \left( \frac{x_n - \lambda}{\sigma_n}\right)^2}$$

Notice that the quantity multiplying the exponential in the above equation is irrelevant because it contributes an overall constant factor to the total likelihood.</br>
Following the computation in the original paper, the maximum value for the log-likelihood is given by:

$$\large \log L^{(k)}_{\max} = b_{k}^{2}/4 a_k$$

where $a_k$ and $b_k$ are defined as follows:

$$ a_k = \frac{1}{2} \sum_n \frac{1}{\sigma^2_n}$$
$$ b_k = \frac{1}{2} \sum_n \frac{x_n}{\sigma^2_n}$$

## Prior Distribution of the Number of Blocks

Following the Bayes rule, we need to implement a prior probability for the number of blocks that are needed. In particular, in most settings it is much more likely a prior for which $N_{blocks} << N$, than $N_{blocks} \approx N$, with $N$ beeing the number of data points. With this in mind, the prior that was adopted is a `geometric prior` with parameter $\gamma$:

$$P(N_{blocks}) = P_0 \gamma^{N_{blocks}} $$

where $P_0$ represent the normalization constant, and it is easily obtained as follows:

$$P(N_{blocks}) = \frac{1-\gamma}{1-\gamma^{N+1}} \gamma^{N_{blocks}}$$
and the expeceted number of blocks is:

$$ <N_{blocks}> = P_0 \sum_{N_{blocks}}^{N} N_{blocks} \gamma^{N_{blocks}} = \frac{N\gamma^{N+1}+1}{\gamma^{N+1}-1} + \frac{1}{1-\gamma}  $$
 
It is possible to note that the estimated number of blocks is a discontiunuos monotonic function of $\gamma$, and for values greater than 1 it leads almost certanily to assigning each datum to a separate block. 

The contribution of the prior to the fitness of each block can be implemented simply by adding the constant $log(\gamma)$ (refered to as `ncp_prior`) to the fitness of each block as follow:

$$F_{total} = \sum_{i=1}^{K} f(B_i) = \sum_{i=1}^{K} log L^{i} + log(\gamma) $$

The choice of $\gamma$ influences the visual representation, adjusting the amount of structure in the blocks, so it is worth to brifly discuss its effects and how to properly choose it. In many application the results are rather insensitive to the value of $\gamma$, as long as the signal to noise ratio is moderatly large, but for extreme value, i.e. ($\gamma \geq$ 1), it can lead to bad results, like too few or too many blocks. The tread off that one must consider when choising the prior is between a conservative choice for the number of blocks, resilient to flactuation but possibly missing real changes, and a liberal choice, which is able to capture more changes and also is more prone to false detections. As suggested by Scargle et al.<a name="cite_note-5"><sup>[5]</sup></a>, an objective method can be implemented as a function of the number of data points $N$, based on the fact that the `ncp_prior` can be seen as a proxy for the false alarm rate $p_0$, which is defined as the frequency with which the algorithm correctlty rejects the presence of a change point. The results of a high number of simulation performed by Scargle et al<a name="cite_note-5"><sup>[5]</sup></a>. for a wide range of $N$ and $p_0$, is that their relation with the `ncp_prior` is well fitted by the following formula:

$$ \text{ncp_prior}  = 4 - log(75.53 p_0 N^{-0.478})$$

## The Algorithm

### Description
Given an ordered set of $N$ data points, the algorithm determines the optimal set of $K + 1$ change-points (and therefore $K$ blocks) which partition the observation intervall,  by iterating through the data points, and caching the current maximum fitness values and
corresponding indices.
At most there can be $N$ total blocks, one for each point.
It can be proven that given $N$ data points the total number of possible partitions of the observation interval is $2^N$. Following the orignal work, it can be shown that removing the last block of an optimal partition leaves an optimal partition. This allows to iteratively consider an increasing number of data points and at each step compute the optimum partition for them, that will be used for the next iteration, thus reducing the space of the possible partition to check drammatically. The computational time of the algorithm is $O(N^2)$.

<center>
<div style='background-color:#f7f7f7; text-align: center; max-width: 1000px; max-height: 1200px; padding-top:30px; padding-left:20px; padding-right:20px; padding-bottom:30px; '>
    <img src="algo.png" width="900" />
</div>
<em><strong>Figure 1</strong>: Representation of the iteration over the possible optimal partition of the observation intervall. The red dots represents the events, while the black ticks over the timeline indicate the possible change points.</em>
</center>

### Our Implementation

For our implementation we have taken inspiration from both the MATLAB code reported in Scargle et al.<a name="cite_note-5"><sup>[5]</sup></a> and from the Astropy source code for the Bayesian Blocks algorithm. 

#### Validate Input

In [ ]:
validate.input = function(data, data_type='Events'){
    
    cells = data
    
    #using the data.frame structure
    if (!is.data.frame(cells)){
        
        cells = as.data.frame(cells)
    }
    
   
    if (data_type=='Events'){
        #count the number of times an event is registered
        cells = cells |> count(cells)
        #sort along times
        return(cells[order(cells[1]), ])
    }
    
    if (data_type=='RegularEvents'){
        # In each tick, there are either zero or one counts. 
        #sort along times
        return(cells[order(cells[1]), ])
    }
    
    if (data_type=='PointMeasures'){
        #sort along times
        return(cells[order(cells[1]), ])
    }
    
}

#### Compute Prior

In [ ]:
compute.p0_prior = function(N, p0=0.05){
    return(4 - log(73.53 * p0 * (N**-0.478))) 
}

In [ ]:
compute.ncp_prior = function(gamma=NULL, N){
    if (is.null(gamma)){
        return(compute.p0_prior(N))
    }
    else{
        return(-log(gamma))
    }
}

#### Compute Fitness

In [ ]:
compute.fitness = function(count_vec, width, dt=NULL, a_k=NULL, b_k=NULL, data_type='Events'){
    
    if (data_type=='Events'){
        return(count_vec*log(count_vec/width))
    }
    else if (data_type=='RegularEvents'){
        M = width/dt
        count_vec_over_M = count_vec/M
        
        esp = 1e-8
        if (sum(count_vec_over_M > 1+esp) > 0){
            cat("regular events: N/M > 1.  Is the time step correct?")
            break
        }
        
        one_m_NM = 1 - count_vec_over_M
        count_vec_over_M[count_vec_over_M <= 0] = 1
        one_m_NM[one_m_NM <= 0] = 1
        
        return(count_vec * log(count_vec_over_M) + (M  - count_vec)*log(one_m_NM))
    }
    else if(data_type=='PointMeasures'){
        return( (b_k*b_k)/(4*a_k) )
    }
    
}

#### Bayesian Blocks

In [ ]:
bayesian_blocks = function(data, ncp_prior=NULL, gamma=NULL, dt=NULL, data_type='Events'){
    
    cells = validate.input(data, data_type)
    
    t = cells[[1]]
    x = cells[[2]]
    if (length(data)==3){sigma = cells[[3]]} #length(data) returns the number of columns of the dataframe 
    else{sigma = 1}
    N = length(t)
    

   
    #quantities needed for the computation of a_k, b_k
    ak_raw = rep(x = 1, times=length(x))/sigma**2
    bk_raw = x/sigma**2
    ck_raw = (x*x)/sigma**2
    
    edges = c(t[1],  0.5 * (t[2:length(t)] + t[1:(length(t)-1)] ), t[length(t)])
    block_length = t[length(t)] - edges
    
    best = rep(0, times = N)
    last = rep(1, times = N)
    
    #-----------------------------------------------------------------
    # Start with first data cell; add one cell at each iteration
    #-----------------------------------------------------------------
    
    for (K in seq(1:N)){
        # Compute the width and count of the final bin for all possible
        # locations of the K^th changepoint
        width = block_length[1:K] - block_length[K + 1] #T_K
        count_vec = rev(cumsum(rev(x[1:K]))) #N_K
        
        a_k = NULL
        b_k = NULL
        
        if (is.null(ncp_prior)){
            ncp_prior = compute.ncp_prior(gamma, N)
        }
        
        if (data_type !='Events' ){
            a_k = 0.5*rev(cumsum(rev(ak_raw[1:K])))
            b_k = -1 * rev(cumsum(rev(bk_raw[1:K])))
        }
        
        
        fit_vec = compute.fitness(count_vec = count_vec, width = width, a_k=a_k, b_k=b_k, dt=dt, data_type)
        
        post_vec = fit_vec - ncp_prior
        
        post_vec[2:length(post_vec)] = post_vec[2:length(post_vec)] + best[1:(K-1)]
        
        i_max = which.max(post_vec)
        last[K] = i_max
        best[K] = post_vec[i_max]
    }
    
    #-----------------------------------------------------------------
    # Recover changepoints by iteratively peeling off the last block
    #-----------------------------------------------------------------

    change_points =  rep(1, times=N)
    i_cp = N
    ind = N+1
    
    while (i_cp>1){
        change_points[i_cp] = ind
        if (ind == 1){ break }
        i_cp = i_cp-1
        ind = last[ind - 1]
    }
    
    if (i_cp==1){change_points[i_cp] = 1}
    change_points = change_points[i_cp:length(change_points)]

    return(edges[change_points])
}

<center>
<div>
<img src="Plots/N2.png" width="1000" />
</div>
<em><strong>Figure 2</strong>: Algorithm computational time test. An increasing number of uniformly distributed events have been generated over the intervall $[0,100)$ and for each one of them have been computed the edges of the bayesian blocks and the time it took to finish the calculations. In order to build some statistics for every data size the algorithm has been run 10 times. The data produced have been fitted with a parabola with its vertex on the origin.</em>
</center>

# Synthetic Data
The BB have been calculated  on `Events` type of data sampled from an uniform and a gaussian distribution, Figure 3. When considering the uniformly distributed samples only one block has been computed that encompass the entirety of the samples. The algorithm perfectly understands the absence of any meaningful structure, yielding the exact pdf that generated the data. As for the gaussian distributed data, the algorithm yield a perfectly reasonable representation of the underlying pdf. 

<center>
    <div>
        <img src="Plots/toy_final.png" width="1200" />
    </div>
    <em><strong>Figure 3</strong>: The first row shows the pdfs that generated the data used to test the algorithm: on the left a uniform distribution over the intervall $[0,100)$ from which have been sampled 1000 points; on the right is plotted a gaussian distribution centered in $0$ and with $\sigma = 0.5$. The second row shows for each pdf two histograms: in red the one built with $\sqrt{N}$ number of fixed width bins, while in blue the one built using the bayesian blocks.
    </em>
</center>

# Prior Dependency
As previously explained, we can inject our knowledge of the data into the algorithm via three different parameter. In particular we explored how changing the value of $\gamma$ in a range from $0$ to $1$ influence the computation of the changing points for a point measures data type. As can be seen from Figure 4, the number of blocks increases as the value  of $\gamma$ grows. When approaching $1$ the obtained representation is heavlily influenced by the data and for $\gamma = 1$ we get one block for each point.

<center>
    <div>
        <img src="gamma_gif.gif" width="1000" />
    </div>
    <em><strong>Figure 4</strong>: The data have been generated by adding a uniformly generated noies, in the range of $[-0.05, 0.05)$, to a signal with a gaussian shape, centered in $3$, with a $\sigma = 1$ and amplitude of $0.5$. The step line represents the bayesian block computed for the different values $\gamma$.
    </em>
</center>

# Bayesian Blocks Vs. KDE
## Kernel Density Estimation
Kernel density estimation (KDE) is a technique for estimation of probability density function that enable the user to have a different view on data that traditionally would be histogrammed.
Unlike the histogram, the kernel technique produces smooth estimate of the pdf, uses all sample points' locations and more convincingly suggest multimodality<a name="cite_note-7"><sup>[7]</sup></a>.


<a name="cite_note-7"></a>7. [^](#cite_note-7)Węglarczyk, Stanisław. "Kernel density estimation and its application." ITM web of conferences. Vol. 23. EDP Sciences, 2018.<br>

## Comparison
Here we report the comparison between the visualizations of some synthetic data obtained with the Bayesian Block algorithm and the kernel density estimation (KDE) method.
As can be seen from Figure 5, even though the KDE representation seems to yield the correct location of the distributions' peaks, it drammatically fails to represent correctly the variance of the most sharp one, moreover the right tail of the distribution falls while the true pdf presents a uniform behaviour.
Meanwhile the bayesian blocks clearly represents the most sharp peak, both its position and variance, while indicating the presence on the wider one, despite beeing less clear with respect to the KDE visualization. The algorithm prefectly shows the uniform distributed reagions.

<center>
<div>
    <img src="Plots/kde_total.png" width="800" />
</div>
    <em><strong>Figure 5</strong>: On the top is plotted the pdf from which the data have been sampled.
        The background is been sampled from a uniform distribution ranging from 0 to 100 for a total of 1000 points, while the two signals consists in 100 points each sampled from two gaussian distribution: the first with $\mu = 10$ and $\sigma=4$, while the second one with $\mu = 40$ and $\sigma=1$
    </em>
</center>

# Applications
Due to the histograms' nature the algorithm can be applied any time this kind of representation is involved. In the following sections we present some results we found and recreate as well as a possible application in deonising some signals. 

## BATSE TTE 

In order to test our implementation of the Bayesian Blocks algorithm we manage to succesfully reproduce the results obtained by Scargle et al.<a name="cite_note-5"><sup>[5]</sup></a>, in section 4.1. In the following figure are reported the Trigger 551 events in the BATSE (Burst & Transient Source Experiment) catalog 4B. The data consists of gamma ray burst photon counts, splitted into energy channels. Even though the data that we use for this plots should be the same from Scargle et al.<a name="cite_note-5"><sup>[5]</sup></a>, we note that the order of magnitude of the Counts is lower than the one reported in the article, but the overall trend is similar.

<center>
    <div>
<img src="Plots/tte_551_plot.png" width="1000" />
        </div>
    <em><strong>Figure 6</strong>: BASTE TTE data for Trigger 551.
    </em>
</center>


## Light Curves
Following the steps of Meyer et al. <a name="cite_note-8"><sup>[8]</sup></a>, we apply the Bayesian Blocks method to $\gamma$-ray light curves obteined by the Fermi Large Area Telescope (LAT)<a name="cite_note-9"><sup>[9]</sup></a>. This data are of the kind `Point Measurements`. The results provide an almost objective way of splitting the light curve into groups of quiescent and flarirng episodes.


<a name="cite_note-8"></a>8. [^](#cite_note-8) Meyer, M., Scargle, J. D., & Blandford, R. D. (2019). Characterizing the gamma-ray variability of the brightest flat spectrum radio quasars observed with the Fermi LAT. The Astrophysical Journal, 877(1), 39. <br>
<a name="cite_note-9"></a>9. [^](#cite_note-9) Data from: https://zenodo.org/record/2598791

<center>
    <div>
<img src="Plots/lc1.png" width="1000" />
        </div>
    <em><strong>Figure 7</strong>: Event 3C273
    </em>
</center>

<center>
    <div>
<img src="Plots/lc2.png" width="1000" />
        </div>
    <em><strong>Figure 8</strong>: Event PKS1510
    </em>
</center>

## ECG
Using the Bayesian Blocks on point measurements type of data we thought to test if was possible to denoise an ECG track without loosing any meaningful information. The data used comes from Kaggle <a name="cite_note-10"><sup>[10]</sup></a>. As can be qualitatively seen by setting `ncp_prior` = 100, thus forcing the representation to use less blocks, the result representation computed by the algorithm perfetcly reproduce the shape of the signal, keeping its key features, while ignoring its small fluctuations and flattening the region with no informations.


<a name="cite_note-10"></a>10. [^](#cite_note-10) Data from: https://www.kaggle.com/datasets/stetelepta/sample-ecg-data?resource=download

<center>
    <div>
<img src="Plots/ecg.png" width="1000" />
        </div>
    <em><strong>Figure 9</strong>: On top is shown the ECG signal while on the botton is plotted the histogramm over the track.
    </em>
</center>

# Conclusion and future work

We succeeded in implementing the bayesian blocks algorithm in R: we tested it on different kind of data, both synthetic and real ones as well of different kinds, and it yielded  insightful representations of them, keeping the key feature of their structures while flattening uniformly distributed regions of the observation space. The algorithm yielded better results in resolving the variance of signals affected by uniform background noise with respect to the KDE method. Its dependency from the prior knowledge, coded as a tunable parameter, results being really useful in order to simplify the data representation while still keeping track of the key element of the underlying distribution. 
Further development on the algorithm will be to implement it in such a way that will enable to study multivariate data as well further investigate for which application this method represent a better visualization with respect to the KDE and vice versa.